In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(filepath_or_buffer="data/resumo/interacoes.csv", header=0)\
    .rename(
        columns={
            "Visitante": "abelha",
            "n° do indivíduo": "id_individuo",
            "Em flor de:": "flor",
        }
    )\
    .drop(['id_individuo'], axis=1)\
    .drop_duplicates(subset=['abelha', 'flor'])

df.to_csv(
    path_or_buf='data/resumo/interacoes-unicas.csv',
    mode='w',
    header=True,
    index=False,
    sep=','
)

df

,abelha,flor
0,Apis mellifera,Andira anthelmia
1,Centris sp.1,Andira anthelmia
2,Centris sp.2,Andira anthelmia
5,Nannotrigona testaceicornis,Andira anthelmia
7,Oxytrigona cagafogo,Andira anthelmia
...,...,...
184,Megachile sp.5,Sphagneticola trilobata
185,Plebeia droryana,Sphagneticola trilobata
187,Ceratina sp.3,Stachytarpheta sp.
188,Paratetrapedia sp.3,Stachytarpheta sp.


Divida os dados em grupos, baseando-se na quantidade

In [3]:
df_count = df\
    .groupby(['flor'])\
    .count()\
    .rename({'abelha': 'count'}, axis=1)\
    .sort_values(by=['count'], ascending=False)

In [4]:
size = 0

for name, row in df_count.iterrows():
    size += row['count']
    if size >= df_count.sum(axis=0).to_list()[0] / 2:
        break

dfs = [
    df[df['flor'].isin(df_count.loc[:name].index.to_list())],
    df[df['flor'].isin(df_count.loc[name:].index.to_list())],
]

Arquivos de matriz de correlação

Escrita do arquivo html

In [5]:
from sankey import SankeyGraph

for i in range(len(dfs)):

    df_interacoes = pd.DataFrame(
        0,
        index=dfs[i]['flor'].drop_duplicates().to_list(),
        columns=dfs[i]['abelha'].drop_duplicates().to_list()
    )

    for _, row in dfs[i].iterrows():
        df_interacoes.at[row['flor'], row['abelha']] = 1

    SankeyGraph(df=df_interacoes)\
        .write_html(
            file=f"data/resumo/interacoes-matriz-correlacao-{i}.html",
            title_text=f"Flores x Abelhas [{i}]",
            font_size=15
        )